## Group 19 (CS986FoMLDAGroup19)
#### Kaggle Team:  Straw-hat Pirates
Peter Hebden (202258273), Afreen Mohsin, Tiasha Mondal (202259828), Shehreen Mushtaq (202289599), Archishman Singha (202285926), Rajesh Bhattacharjee (202264331)

# Spotify Regression: predict song popularity. 

This dataset includes these features:  
**Id**: unique track identifier.  
**title**: track title.  
**artist**: singer or band.  
**year**:  year of release (or re-release).  
**bpm**:  beats per minute.  
**nrgy**:  energy: higher, more energetic.  
**dnce**: danceability: higher, the easier to dance to.  
**dB**:  loudness (dB): the higher the value, the louder the song.  
**live**: liveness: higher, more likely it's a live recording.  
**val**: valence: higher, more positive mood.  
**dur**: duration: song length.  
**acous**: acousticness: higher, more acoustic.  
**spch**: speechiness: higher, more spoken word.  
**pop**:  popularity: higher is more popular.  
**top genre**:  genre of the track (class label).  

Spotify: https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features  

Kaggle data: https://www.kaggle.com/cnic92/spotify-past-decades-songs-50s10s.   
The training dataset contains 453 rows × 15 columns including the Id column. There are 13 column attributes in the dataset that can be used to build a model that predicts "popularity" (pop).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# default is 120 seconds
%autosave 60 

# Load the data

Load the train and Kaggle test data: the train data will be split into train and validation sets for hyperparameter tuning and to select the best model. The best model will be trained on the entire training set and used to make predictions using the test set for submission to Kaggle.

In [ ]:
# load dataset
on_kaggle=False
    
if on_kaggle:
    dataset=pd.read_csv('/kaggle/input/cs9856-spotify-regression-problem-2023/CS98XRegressionTrain.csv')
    test_dataset=pd.read_csv("/kaggle/input/cs9856-spotify-regression-problem-2023/CS98XRegressionTest.csv")
else:
    dataset=pd.read_csv('CS98XRegressionTrain.csv')
    test_dataset=pd.read_csv("CS98XRegressionTest.csv")

In [ ]:
dataset.columns

# EDA and Visualization

### Data summary
For this task our first step was to perform exporatory data analysis (EDA) and visualization.
**train_dataset.describe()** shows a summary of the numerical training data. 
The range of values for each feature appears to be valid. For example, the range 
for bpm is 62 to 199 with a mean and median close to 120bpm, which is considered to be 
optimal for song popularity.  **train_dataset.info()** shows that top genre 
is 15 missing values (453-438). The missing top genre values were replace by the mode (adult standards). Next,  **value_counts() and plt.bar()** revealed 86 genres, only 1 or 2 examples for most genres, but 
that was not an issue because we were trying to predict a song popularity based mostly on numerical data.

### Correlation
Multicollinearity: where independent variables are strongly correlated. This can reduce model performance. The variation inflation factor (VIF) for each variable can be computed. If predictive features yield VIF values less than 5, then this would indicate that multicollinearity is not a problem. However, the ***VIF*** output below shows that bpm and dnce are over 20!

The correlation matrix below shows that dB and nrgy have the highest positively correlated features, val and dnce are second, while dnce and acoustics are the most negatively correlated. However, most of the features in this dataset have low correlation and, in fact, the application of PCA did not improve classification accuracy.

### Outliers
Next we used ***sns.boxplot()*** to reveal potential outliers. The default value for the whiskers is 1.5 standard deviations. The boxplots show that some features include data points far beyond the whiskers. While it's difficult to distinguish between significant data points and outliers (noise), we found dropping rows or setting outlier values to 0 did not reduce RMSE. And the threshold made little because there were few obvious cases of outliers to begin with.

### Clusters
t-SNE can project higher dimensional data onto a 2D space such that points that are relatively close in high dimensional space are close in 2D space.  The t-SNE plot below indicates that the songs cluster in a nonrandom way and structure in the data that may be exploited for regression.

### What worked and what did not
Most of the things we tried made either a small incremental improvement or none.  Dropping attributes (columns) usually reduced pop prediction errors (increased RMSE). PCA made little difference. Tuning model hyperparameters made little difference. But one-hot encoding title, artist, year and genre had a positive although modest impact on regression accuracy. While dropping any of the features increased validation RMSE, we concluded that the one-hot encode features contributed to overfitting.


Data pre-processing will be discussed after EDA and visualization.


## Data exploration

In [ ]:
##copy of the original dataset
train_dataset = dataset.copy()
#creating a copy of the test dataset
test_df = test_dataset.copy()

### Distribution of song popularity

There are 59 unique pop scores in the training set. The distribution of song counts per unique score is very skewed. There are many low pop songs (pop < 30) and few very pop songs (pop > 50). So our methods should not assume the the target variable is normally distributed.

In [ ]:
y_all = train_dataset["pop"]
value_counts=y_all.value_counts()
print("number of c:", len(value_counts))

In [ ]:
num_bins=len(value_counts)

x=[x for x in range(1,num_bins+1)]
# full training set is very unbalanced, 
# most genres have only 1 or 2 examples
plt.figure(figsize=(20,3)) 
value_counts.hist(bins=num_bins+1); 
plt.bar(x, value_counts.values);
#plt.xticks(rotation=90);
plt.xlabel('popularity')
plt.ylabel('count');
plt.title('Song Popularity')

### Correlation
While some independent variable were relatively highly correlated, nrgy and dB in particular (0.68), 
selecting subsets of variable based on correlation was not helpful. Also, none of the independent 
variables were highly correlated with pop.

In [ ]:
# Plot linear correlation matrix
cm_data=train_dataset.copy()
cm=cm_data.iloc[:,1:14].corr()   # make sure indices are right!
#cm[cm<0.45]=0

fig, ax = plt.subplots(figsize=(9,3))
sns.heatmap(cm, annot=True, cmap='YlGnBu', vmin=-1,
vmax=1, center=0, ax=ax)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
train_dataset

### VIF

We calculated the variance inflation factor for each feature. The equation: **VIF = 1/(1-R^2)** where R^2 is the coefficient of determination. In general, if a feature that has a VIF value greater than 5 is considered to be highly collinear with other features in the data.  The output below shows that bpm and dnce are over 20!  Unfortunately, dropping these features increased RMSE. 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_data=train_dataset.iloc[:,5:14].copy()   # if this fails, the indices are wrong
VIF             = pd.DataFrame()
VIF['feature']  = num_data.columns
VIF['VIF']      = [variance_inflation_factor(num_data.values, i) for i in range(num_data.shape[1])]
VIF

### Boxplots show outliers

In [ ]:
#boxplots: visualize data distributions and outliers

plt.figure(figsize=(7,5));
df=train_dataset.drop("top genre", axis=1, inplace=False)
attributes=df.columns.values.tolist()
idx=4
for attribute in attributes[idx:]:
    plt.subplot(5,3,idx)
    sns.boxplot(x=train_dataset[attribute])
    idx=idx+1
    plt.tight_layout(pad=1)

### t-SNE shows some clustering in the training data
Songs with very low popularity (black or very dark colors) are mostly in two groups.
Medium and high popularity songs (medim and light colors) are in many small, widely dispersed groups.
These small groups could be clusters of similar genres.

In [ ]:
show_legend=False

# visualize training data
temp_data=dataset.copy()
#temp_data.dropna(axis=0, inplace=True)
temp_data.drop(['title','artist', 'top genre'], axis=1, inplace=True)
df = pd.DataFrame()
df["y"] = temp_data['pop'].copy();
y_pop=df["y"].sort_values(ascending=False)
#temp_data.drop(['pop'], axis=1, inplace=True)


import numpy as np
from sklearn.manifold import TSNE

X_train_embedded = TSNE(n_components=2, learning_rate='auto',
                init='random', perplexity=3).fit_transform(temp_data)
X_train_embedded.shape

df["comp-1"] = X_train_embedded[:,0]
df["comp-2"] = X_train_embedded[:,1]

# 59 unique pop scores, must be done after drop NA
n_colors=len(df["y"].unique()) 
plt.figure(figsize=(5,4));
T=sns.scatterplot(x="comp-1", y="comp-2", hue=y_pop, legend=show_legend,\
                palette=sns.color_palette("rocket", n_colors), \
                data=df, alpha=.9);
plt.title('t-SNE projection: lighter indicates more popular', fontsize=10)

if show_legend:
    plt.legend(bbox_to_anchor=(1.4, 1), loc='upper right', borderaxespad=0);

# Data Pre Processing Steps on Train Set

### Categorical variable were one-hot encoded

Based on data exporation and analysis, we decided to one-hot encode some of the categorical features on the theory that they contained some predictive information. Intuitively, we suspected that year is at least weakly associated with song popularity, while genre artist would be more strongly associated with popularity. One-hot encoding of categorical data resulted in a small reduction of RMSE.

In [ ]:
dataset["top genre"].fillna('adult standards', inplace=True)

In [ ]:
# dropping the columns not required from train dataset
dataset = dataset.drop(["Id"], axis=1)
#dropping the pop column for test set
test_dataset = test_dataset.drop(["Id"], axis=1)

In [ ]:
##copy of the original dataset
train_dataset = dataset.copy()

In [ ]:
#creating top genre in the test dataset
c = "pop"
test_dataset = test_dataset.assign(**{c: pd.Series(dtype='object')})

In [ ]:
#concating the train and test dataset
concatenated_dataset = pd.concat([dataset, test_dataset])

In [ ]:
#give the same column name as the original dataset 
concatenated_dataset = concatenated_dataset.reset_index(drop=True)
concatenated_dataset.columns= train_dataset.columns

In [ ]:
#dividing the dataset into dependable and independent variables
x_con_data=concatenated_dataset.iloc[:,:-1]
y_con_data=concatenated_dataset.iloc[:,-1]

### One-hot encoding

In [ ]:
#one hot encoding

# select the categorical variables
categorical_variables = ['title','artist','year', 'top genre']

# perform one hot encoding
x_con_data = pd.get_dummies(x_con_data, columns=categorical_variables)

In [ ]:
#adding the top genre in the one hot encoded dataset
x_con_data['pop'] = y_con_data
#assigning the data to the variable concatenated data
concatenated_dataset = x_con_data

### Split the concatenated dataset back into train and kaggle test sets.

In [ ]:
# Manually specify the row index to split the dataset
split_index = train_dataset.shape[0]

# Split the dataset into two parts
dataset = concatenated_dataset.iloc[:split_index]
test_dataset = concatenated_dataset.iloc[split_index:]

# Print the size of the training and testing datasets
print(f"Training dataset size: {np.shape(dataset)}")
print(f"Testing dataset size: {np.shape(test_dataset)}")

### Detect missing values and make records

In [ ]:
#checking for null values in each columns in the dataset
print(sum(dataset.iloc[:,:-1].isnull().any()))
print(dataset.iloc[:,-1].isnull().any())

In [ ]:
from sklearn.model_selection import train_test_split
# Create a boolean mask of rows with missing values
mask = dataset.isnull().any(axis=1)

# Split the data into two sets: with and without missing values
test_set = dataset[mask]
train_set = dataset[~mask]

In [ ]:
#extracting the pop column
pop_train=train_set["pop"]
pop_test=test_set["pop"]
pop = np.concatenate((pop_train,pop_test), axis=0)

# dropping the columns not required from train datase
train_set = train_set.drop(["pop"], axis=1)
test_set = test_set.drop(["pop"], axis=1)

In [ ]:
# Split the set without missing values into training and test sets
x_train = train_set
y_train = train_set#["top genre"]
x_test = test_set#.drop("top genre",axis=1)
y_test = test_set#["top genre"]

In [ ]:
#copy the x_train dataset and check for null values
train_dataset = x_train.copy()
print(sum(x_train.isnull().any()))
print(sum(y_train.isnull().any()==True))
#checking for null values in x_test and y_test
print(sum(x_test.isnull().any()))
print(sum(y_test.isnull().any()==True)) # True so handle missing vals next

In [ ]:
# Concatenate x_train and x_test
x = np.concatenate((x_train, x_test), axis=0)
# Concatenate y_train and y_test
y = np.concatenate((y_train, y_test), axis=0)
# Convert x_train and y_train to data frames
x = pd.DataFrame(x)
y = pd.DataFrame(y)

In [ ]:
# #adding the pop column back into the train dataset
dataset['pop'] = pop
train_dataset = dataset.copy()
#concating the train and test dataset
concatenated_dataset = pd.concat([dataset, test_dataset])
#give the same column name as the original dataset 
concatenated_dataset = concatenated_dataset.reset_index(drop=True)
concatenated_dataset.columns= train_dataset.columns
#concatenated_dataset
#dividing the dataset into independent and dependent variables
x_con_data=concatenated_dataset.iloc[:,:-1]
y_con_data=concatenated_dataset.iloc[:,-1]

### One-hot encoding

In [ ]:
#adding the top genre in the one hot encoded dataset
x_con_data['pop'] = y_con_data
#assigning the data to the variable concatenated data
concatenated_dataset = x_con_data

# Manually specify the row index to split the dataset
split_index = 453

# Split the dataset into two parts
dataset = concatenated_dataset.iloc[:split_index]
test_dataset = concatenated_dataset.iloc[split_index:]

# Print the size of the training and testing datasets
print(f"Training dataset size: {np.shape(dataset)}")
print(f"Testing dataset size: {np.shape(test_dataset)}")
test_dataset.drop(["pop"], axis=1, inplace=True) # drop place holder column

### Check for duplicates
One found and deleted.

In [ ]:
# Check for duplicates
duplicates = dataset.duplicated().sum()
print(f"Number of duplicates: {duplicates}")
dataset = dataset.drop_duplicates()

### Standardize and process outliers

In [ ]:
# Define function to handle outliers using z-score method
def handle_outliers_z_score(df, threshold=3):
    z_scores = np.abs((df - df.mean()) / df.std())
    df_out = df[(z_scores < threshold).all(axis=1)]
    return df_out

# Apply function to relevant columns
outliers_dataset = dataset.copy()
columns = ['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch']
df_outliers_removed = handle_outliers_z_score(outliers_dataset[columns])
dataset = pd.concat([df_outliers_removed, dataset.drop(columns, axis=1) ], axis=1)
dataset = dataset.dropna()

# Print number of rows before and after handling outliers
print(f"Original number of rows: {len(outliers_dataset)}")
print(f"Number of rows after handling outliers: {len(dataset)}")

In [ ]:
scaler=StandardScaler()
scaler.fit(dataset.iloc[:,:9])
dataset.iloc[:,:9]=scaler.transform(dataset.iloc[:,:9])

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [ ]:
#dividing the dataset into independent and dependent variables
X_train=dataset.iloc[:,:-1]
Y_train=dataset.iloc[:,-1]

# Train the models

In this section we trained the Linear Regression (LR), Support Vector Regression (SVR), and Random Forest Regressor (RFR) machine learning models on the training data. Support vector machines are well suited to small and medium sized complex data datasets. Random Forests and XGBoost known for excellent performance and we would try XGBoost in the future. Nonetheless, the quantity and quality (i.e. before and after pre-processing) of the dataset had a larger impact on performance than choice of model.

We used GridSearchCV to find the best hyperparameters and perform cross validation to identify the best model. We selected the best model based on root mean squared error (RMSE) on validation data. The three models delivered similar results, although the RFR model was relatively slow due to running many decision trees.  

Linear Regression (LR) was chosen as a benchmark and based on the idea that one should try a simple model first. Indeed, simple models are fast and often perform surprisngly well on tabluar data. It's training RMSE was virtually zero and yet it's best validation RMSE was about 11.057 and not signicantly different from SVR and RFR.  


SVR is a more complex model and we hoped that it would deliver a low RMSE. While SVR gets the best validation at the moment, the RMSE diffrence is small. The plot below shows that the validation RMSE is almost flat, i.e. decreasing regularization had little impact on RMSE.  The train RMSE mainatined a strong downward trajectory as C increased (and regularization decreased), a sign of overfitting.

RFR uses many decision tree estimators (can be hundreds) to predict real values. This model relies on the wisdom of crowds concept where many weak estimators combine to make accurate predictions. The number of estimators can be optimized, but we expected that ***max_depth*** would be the most important hyperparameter.  However, parameter tuning did not have a signicant impact on validation RMSE.

The train and validation plots for SVR and Random Forest are strikingly similar and their validation RMSE's are flat. Very little generalization. It seems that our one-hot code dataset is not well suited to this regression task.  In contrast, we found that one-hot encoding greatly improved classification accuracy (in our other jupyter notebook).

Validation and selection were done prior running the model on the unseen test data for submission to Kaggle. In general, we concluded our models were overfitting the data. However, we also concluded that one-hot encoding of categorical data in this dataset added noise to the dataset and it would have been better to focus on pre-processing the numerical features.

### Summary of final results
Utimately, our top scoring model on Kaggle was a RandomForest Regressor, with a RMSE of 7.44924 (much lower than on our validation RMSE).  Group name: Straw-hat Pirates, position 25 on the Kaggle leaderboard.

#### LR_model = LinearRegression(fit_intercept=True, random_state=42)
#### SVR_model = SVR(C=12, degree=2, kernel='rbf', random_state=42)
#### RF_model = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42)

Typical and reprodicible results are shown in this table and the plots below.

| Model | Train RMSE | Validation RMSE |
| --- | --- | --- |
| Linear Regression | ~0    | 11.057 |
| Support Vector Regression | 5.367 |  10.99 |
| Random Forest Regressor   | 7.137 |  11.280 |

### Feature selection and PCA dimensionality reduction
These methods and more were tied but RMSE remained high. It seems that more domain knowledge is needed for better data pre-processing.

In [ ]:
features=['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch']
features[0:9]
ft=[2,3,4,5]

In [ ]:
# apply PCA to entire training set and capture ~95% of the variance.
from sklearn.decomposition import PCA
pca=PCA(n_components = 300)
X_train_pca = pca.fit_transform(X_train)  
#print(pca.explained_variance_ratio_) # depends on app, but need 80 to 95% of variance
print("total explained variance by n components: {:0.2f}%".format(sum(pca.explained_variance_ratio_)*100))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create a linear regression model
model = LinearRegression()  # no random_state to set!

# Define the hyperparameters to tune and their values to test
param_grid = {'fit_intercept': [True]}

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
grid_search.fit(X_train, Y_train)

best_params = grid_search.best_params_
print(best_params)
best_score = grid_search.best_score_

rmse_train = np.sqrt(-1*grid_search.cv_results_['mean_train_score'])
rmse_val = np.sqrt(-1* grid_search.cv_results_['mean_test_score'])

print("RMSE weighted train score={}".format(rmse_train))
print("RMSE weighted validation score={}".format(rmse_val))
print("best validation score={}".format(np.sqrt(-1*best_score)))

In [ ]:
# train on all training data, to run model on real test data
LR_model = LinearRegression(fit_intercept=True)
LR_model.fit(X_train, Y_train)

# Support Vector Regression (SVR)

In [ ]:
from sklearn.svm import SVR
# Create an SVR model
svr_model = SVR( ) # no random state
# Define the hyperparameter grid to search over
param_grid = {'C': [1,2,3,4,5,6,7,8,9,10,11,12],'kernel': ['rbf'],
              'degree': [2]}

# Create a GridSearchCV object with the SVR model and hyperparameter grid
grid = GridSearchCV(svr_model, param_grid, scoring='neg_mean_squared_error',\
                    cv=5, return_train_score=True)

# Fit the GridSearchCV object to the training data
grid.fit(X_train, Y_train)

# Get the best hyperparameters
best_params = grid.best_params_
print("best params", best_params)
best_score = np.sqrt(-1 *grid.best_score_)

rmse_train= np.sqrt(-1 * grid.cv_results_['mean_train_score'])
rmse_val  = np.sqrt(-1 * grid.cv_results_['mean_test_score'])

print("RMSE weighted train score={}".format(rmse_train))
print("RMSE weighted validation score={}".format(rmse_val))
print("best validation score={}".format(best_score))

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(rmse_train, label="train")
plt.plot(rmse_val, label="validation")
plt.title("Support Vector Regressor")
plt.ylabel("RMSE")
plt.xlabel("C")
plt.legend();

In [ ]:
# train on ful training set to run on Kaggle test data
SVR_model = SVR(C=best_params['C'], kernel=best_params['kernel'], \
                degree=best_params['degree'])
SVR_model.fit(X_train, Y_train)

## Random Forest Regressor

In [ ]:
# train on all training data, to run model on real test data
from sklearn.ensemble import RandomForestRegressor

C_vec=np.arange(1,11,1)

rf_model = RandomForestRegressor(random_state=42)
# Define the hyperparameter grid to search over
param_grid = { 
    'n_estimators': [100],
    'max_depth' : [3,6,9,12]
}

# Create a GridSearchCV object with the SVR model and hyperparameter grid
grid = GridSearchCV(rf_model, param_grid, scoring='neg_mean_squared_error',\
                    cv=5, return_train_score=True)
# Fit the GridSearchCV object to the training data
grid.fit(X_train_pca, Y_train)
# Get the best hyperparameters
best_params = grid.best_params_
print("best params", best_params)

rmse_train= np.sqrt(-1 * grid.cv_results_['mean_train_score'])
rmse_val  = np.sqrt(-1 * grid.cv_results_['mean_test_score'])

print("RMSE weighted train scores={}".format(rmse_train))
print("RMSE weighted validation scores={}".format(rmse_val))

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(rmse_train, label="train")
plt.plot(rmse_val, label="validation")
plt.title("Random Forest Regressor")
plt.ylabel("RMSE")
plt.xlabel("max depth")
plt.legend();

In [ ]:
# train on all training data, to run model on real test data
RF_model = RandomForestRegressor(max_depth=best_params['max_depth'],\
                                 n_estimators=best_params['n_estimators'], random_state=42)
RF_model.fit(X_train, Y_train)  

# Test Data Prediction

In [ ]:
#feature scaling on test dataset
test_dataset_arr=test_dataset.copy()
test_dataset_arr.iloc[:,:9]=scaler.transform(test_dataset_arr.iloc[:,:9]) 

In [ ]:
#
# Select model to test and automatically save predictions to csv file.
#
linear_regression=True
svm_regression=False
rf_regression=False

Y_test_done=False

if linear_regression:
    Y_test = LR_model.predict(test_dataset_arr)
    file_name="y_pred_LRR.csv"
    Y_test_done=True
elif svm_regression:
    Y_test = SVR_model.predict(test_dataset_arr)
    file_name="y_pred_SVR.csv"
    Y_test_done=True
elif rf_regression:   
    Y_test = RF_model.predict(test_dataset_arr)
    file_name="y_pred_RFR.csv"
    Y_test_done=True

In [ ]:
# save predictions to csv file for uploading to kaggle
def save_y_pred(X_TEST_Id, y_pred_test, file_name):
  with open(file_name, 'w') as f:
    f.write("Id," + "pop\n")
    for ii in range(len(y_pred_test)):
      f.write(str(X_TEST_Id[ii]) + ",")
      f.write(str(y_pred_test[ii])+"\n")

In [ ]:
if Y_test_done:
    TEST_Id= test_df.iloc[:,:1].values
    TEST_Id=TEST_Id.flatten()
    TEST_Id=list(TEST_Id)
    y_test_pred=list(Y_test)
    save_y_pred(TEST_Id, y_test_pred, file_name)